#### Extracción tabla datos Selenium

In [2]:
import pandas as pd
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')
from selenium.webdriver.common.by import By
from IPython.display import Image # Para mostrar imágenes en python
from bs4 import BeautifulSoup as bs
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

pd.set_option('display.max_columns', None)

In [2]:
from selenium import webdriver
PATH = 'driver/chromedriver.exe'
driver = webdriver.Chrome()

Cache folder (C:\Users\jacom\.cache\selenium) cannot be created: No se puede crear un archivo que ya existe. (os error 183)
Cache folder (C:\Users\jacom\.cache\selenium) cannot be created: No se puede crear un archivo que ya existe. (os error 183)


In [3]:
url = 'https://www.portsdebalears.com/es/buques-en-puerto' # Definimos url de búsqueda

In [4]:
driver.get(url)

In [5]:
driver.find_element(By.XPATH, '//*[@id="popup-buttons"]/button[1]').click() # Clickamos en cookies

In [6]:
# Ojo, la tabla que queremos extraer está dentro de un frame distinto al de la página principal (por eso la url no cambia al buscar tablas de datos diferentes)

driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="giswebframe"]'))

In [7]:
driver.find_element(By.ID, 'trafico_menu').click() # Ahora si podemos clickar sobre los datos históricos

In [8]:
# Me defino una función que me haga el triple click

actions = ActionChains(driver)
def triple_click(element_x):
    actions.click(element_x).click(element_x).click(element_x).perform()

In [ ]:
/html/body/div[4]/div[2]/div[3]/div/div[1]/div[2]/input
/html/body/div[4]/div[2]/div[3]/div/div[1]/div[4]/input

In [9]:
# Nuestro próximo reto es introducir las fechas que queremos de rango
# Tendremos que clickar en la caja y, mejor que clickar sobre fecha, introducir por texto el rango de fechas que nos interesa

fecha_ini = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[3]/div/div[1]/div[2]/input') # Apunto con el full XPATH
triple_click(fecha_ini)
fecha_ini.send_keys("01/06/2023") # Meto fecha entrada
time.sleep(2)

fecha_fin = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[3]/div/div[1]/div[4]/input') # Apunto con el full XPATH
triple_click(fecha_fin)
fecha_fin.send_keys("31/08/2023") # Meto fecha salida
time.sleep(2)
driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[3]/div/div[1]/div[5]/button[1]/span').click()
time.sleep(10)

In [ ]:
/html/body/div[4]/div[8]/div/div[2]/div/div/div/div[2]/div[3]/div[3]/div/table/tbody

Habiendo introducido las fechas vamos a obtener la tabla de datos de transito de buques

In [26]:
time.sleep(5)
tabla = driver.find_element(By.XPATH, '/html/body/div[4]/div[8]/div/div[2]/div/div/div/div[2]/div[3]/div[3]/div/table/tbody') # Asignamos variable tabla al cuerpo de la tabla
filas = tabla.find_elements(By.TAG_NAME, 'tr') # Cada fila está metida en un 'tr'

data = []

for f in filas:

    elementos = f.find_elements(By.TAG_NAME, 'td') # Cada fila está metida en un 'td'

    tmp = []

    for e in elementos:
        tmp.append(e.text)

    data.append(tmp)

In [27]:
df = pd.DataFrame(data)

In [30]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,,,,,,,,,,VALIANT LADY,,,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192,BAHAMAS,277,8,,,,,,,,,,,,,,
2,,,,,,,,,,ELEANOR ROOSEVELT,,,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066,CHIPRE,123,4,,,,,,,,,,,,,,
3,,,,,,,,,,TENACIA,,,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462,ITALIA,199,6,,,,,,,,,,,,,,
4,,,,,,,,,,HEDY LAMARR,,,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422,CHIPRE,186,7,,,,,,,,,,,,,,


In [31]:
df_transito = df.copy()

In [32]:
df_transito.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,,,,,,,,,,VALIANT LADY,,,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192,BAHAMAS,277,8,,,,,,,,,,,,,,
2,,,,,,,,,,ELEANOR ROOSEVELT,,,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066,CHIPRE,123,4,,,,,,,,,,,,,,
3,,,,,,,,,,TENACIA,,,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462,ITALIA,199,6,,,,,,,,,,,,,,
4,,,,,,,,,,HEDY LAMARR,,,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422,CHIPRE,186,7,,,,,,,,,,,,,,


In [33]:
df_transito.shape

(1050, 38)

In [34]:
df_transito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 38 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1050 non-null   object
 1   1       1050 non-null   object
 2   2       1050 non-null   object
 3   3       1050 non-null   object
 4   4       1050 non-null   object
 5   5       1050 non-null   object
 6   6       1050 non-null   object
 7   7       1050 non-null   object
 8   8       1050 non-null   object
 9   9       1050 non-null   object
 10  10      1050 non-null   object
 11  11      1050 non-null   object
 12  12      1050 non-null   object
 13  13      1050 non-null   object
 14  14      1050 non-null   object
 15  15      1050 non-null   object
 16  16      1050 non-null   object
 17  17      1050 non-null   object
 18  18      1050 non-null   object
 19  19      1050 non-null   object
 20  20      1050 non-null   object
 21  21      1050 non-null   object
 22  22      1050 non-null   

In [35]:
df_transito[9].unique() # Nombres de barcos únicos

array(['', 'VALIANT LADY', 'ELEANOR ROOSEVELT', 'TENACIA', 'HEDY LAMARR',
       'HYPATIA DE ALEJANDRIA', 'CIUDAD DE PALMA', 'CIUDAD DE ALCUDIA',
       'KERRY', 'GNV BRIDGE', 'COSTA DIADEMA', 'SEVEN SEAS MARINER',
       'ECO ADRIATICA', 'MSC SEAVIEW', 'NORWEGIAN EPIC',
       'MARELLA VOYAGER', 'AIDACOSMA', 'CEMBROOK', 'HAPPY DELTA',
       'GNV SEALAND', 'GUBAL TRADER', 'MEIN SCHIFF 2', 'COSTA FASCINOSA',
       'GYPTIS', 'CIUDAD DE GRANADA', 'TRITON', 'SYMPHONY OF THE SEAS',
       'ECO MEDITERRANEA', 'MARELLA DISCOVERY 2', 'AIDASTELLA',
       'CELEBRITY EDGE', 'MSC ORCHESTRA', 'MV CPC CAROLINA', 'TARIFA',
       'MARTIN I SOLER', 'BBC MOONSTONE', 'EUROPA', 'SPIRIT OF DISCOVERY',
       'CIUDAD DE BARCELONA', 'SYMPHONY SPIRIT', 'FORMENTOR',
       'CARABINIERE', 'ABSA UNO', 'ARVIA', 'BIO LAS PALMAS', 'FUROR',
       'GUNGOR DURMUS', 'CASTILLA', 'USS JAMES E WILLIAMS', 'PATIÑO',
       'CANARIAS', 'SIRIOS CEMENT V', 'BAHAMA MAMA', 'FOSCARI',
       'GOLDEN BRIDGE', 'NORWEGIAN GETAW

In [36]:
len(df_transito[9].unique())

90

90 barcos únicos de entre 1050 incidencias de tránsito.

In [37]:
df_transito[18].unique() # Empresas dueñas de los barcos 

array(['', 'ESTELA SHIPPING PALMA SA',
       'BALEARIA EUROLINEAS MARITIMAS S.A.U.',
       'TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU',
       'GRIMALDI LOGISTICA ESPAÑA, SL',
       'INTERCRUISES SHORESIDE & PORT SERVICES', 'MIGUEL PUIGSERVER S A',
       'MARITIMA DEL MEDITERRANEO SA', 'BALEARES CONSIGNATARIOS S L',
       'LANTIMAR, S .L.', 'A PEREZ Y CIA SL'], dtype=object)

In [38]:
len(df_transito[18].unique())

11

De las 1050 incidencias de barcos, todos se pueden catalogar en 11 empresas.

#### Limpieza dataframe

In [39]:
# Podemos borrar con toda tranquilidad las columnas [0:8] y [24::] porque no contienen información.

df_transito = df_transito.drop(df_transito.columns[0:9], axis=1) 

In [40]:
df_transito

,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,VALIANT LADY,,,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192,BAHAMAS,277,8,,,,,,,,,,,,,,
2,ELEANOR ROOSEVELT,,,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066,CHIPRE,123,4,,,,,,,,,,,,,,
3,TENACIA,,,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462,ITALIA,199,6,,,,,,,,,,,,,,
4,HEDY LAMARR,,,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422,CHIPRE,186,7,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,CIUDAD DE ALCUDIA,,,Valencia,Valencia,31/08/2023 06:57,31/08/2023 11:25,,1ER. TRAMO EXT MLLES CIALES. (del 12 al 2),"GRIMALDI LOGISTICA ESPAÑA, SL",33.588,3398,ESPAÑA,183,7,,,,,,,,,,,,,,
1046,YACHT SERVANT,,,GENOA,PORT EVERGLADES,31/08/2023 08:57,02/09/2023 16:05,,M. 2ª ALIN. PONIENTE SUR (del 18 al 2),MIGUEL PUIGSERVER S A,44.522,3408,PAÍSES BAJOS,214,5,,,,,,,,,,,,,,
1047,ABSA UNO,,,SAN PEDRO DEL PINATAR,ALMERIA,31/08/2023 10:30,31/08/2023 18:50,,MUELLE ADOSADO (del 16 al 12),ESTELA SHIPPING PALMA SA,317,3353,ESPAÑA,37,5,,,,,,,,,,,,,,
1048,ECO ADRIATICA,,,CAGLIARI,SAGUNTO,31/08/2023 13:30,31/08/2023 15:28,,1ER. TRAMO EXT MLLES CIALES. (del 15 al 2),"GRIMALDI LOGISTICA ESPAÑA, SL",67.311,3338,ITALIA,238,7,,,,,,,,,,,,,,


In [46]:
df_transito.reset_index(drop=True)

,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,,,,,,,,,,,,,,,,,,,,,,,,
1,VALIANT LADY,,,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192,BAHAMAS,277,8,,,,,,,,,
2,ELEANOR ROOSEVELT,,,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066,CHIPRE,123,4,,,,,,,,,
3,TENACIA,,,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462,ITALIA,199,6,,,,,,,,,
4,HEDY LAMARR,,,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422,CHIPRE,186,7,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,CIUDAD DE ALCUDIA,,,Valencia,Valencia,31/08/2023 06:57,31/08/2023 11:25,,1ER. TRAMO EXT MLLES CIALES. (del 12 al 2),"GRIMALDI LOGISTICA ESPAÑA, SL",33.588,3398,ESPAÑA,183,7,,,,,,,,,
1046,YACHT SERVANT,,,GENOA,PORT EVERGLADES,31/08/2023 08:57,02/09/2023 16:05,,M. 2ª ALIN. PONIENTE SUR (del 18 al 2),MIGUEL PUIGSERVER S A,44.522,3408,PAÍSES BAJOS,214,5,,,,,,,,,
1047,ABSA UNO,,,SAN PEDRO DEL PINATAR,ALMERIA,31/08/2023 10:30,31/08/2023 18:50,,MUELLE ADOSADO (del 16 al 12),ESTELA SHIPPING PALMA SA,317,3353,ESPAÑA,37,5,,,,,,,,,
1048,ECO ADRIATICA,,,CAGLIARI,SAGUNTO,31/08/2023 13:30,31/08/2023 15:28,,1ER. TRAMO EXT MLLES CIALES. (del 15 al 2),"GRIMALDI LOGISTICA ESPAÑA, SL",67.311,3338,ITALIA,238,7,,,,,,,,,


In [62]:
df_transito.columns = [str(i) for i in range(24)]

In [63]:
df_transito.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'],
      dtype='object')

In [73]:
df_transito.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,,,,,,,,,,,,,,,,,,,,,,,,
1,VALIANT LADY,,,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192,BAHAMAS,277,8,,,,,,,,,
2,ELEANOR ROOSEVELT,,,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066,CHIPRE,123,4,,,,,,,,,
3,TENACIA,,,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462,ITALIA,199,6,,,,,,,,,
4,HEDY LAMARR,,,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422,CHIPRE,186,7,,,,,,,,,


In [84]:
# Voy a crear una lista con las columnas que quiero borrar y hago drop de ellas
df_transito.drop(columns = ['23'], inplace=True)
df_transito.head()



,0,3,4,5,6,8,9,10,11,12,13,14
0,,,,,,,,,,,,
1,VALIANT LADY,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192,BAHAMAS,277,8
2,ELEANOR ROOSEVELT,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066,CHIPRE,123,4
3,TENACIA,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462,ITALIA,199,6
4,HEDY LAMARR,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422,CHIPRE,186,7


In [85]:
df_transito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1050 non-null   object
 1   3       1050 non-null   object
 2   4       1050 non-null   object
 3   5       1050 non-null   object
 4   6       1050 non-null   object
 5   8       1050 non-null   object
 6   9       1050 non-null   object
 7   10      1050 non-null   object
 8   11      1050 non-null   object
 9   12      1050 non-null   object
 10  13      1050 non-null   object
 11  14      1050 non-null   object
dtypes: object(12)
memory usage: 98.6+ KB


In [88]:
# Voy a renombrar las columnas a sus respectivos heads
heads = ['barco', 'origen', 'destino', 'llegada', 'salida', 'alineacion', 'consignatorio', 'gt','escala', 'bandera', 'eslora', 'calado']

In [89]:
df_transito.columns = heads

In [90]:
df_transito.head()

,barco,origen,destino,llegada,salida,alineacion,consignatorio,gt,escala,bandera,eslora,calado
0,,,,,,,,,,,,
1,VALIANT LADY,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192,BAHAMAS,277,8
2,ELEANOR ROOSEVELT,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066,CHIPRE,123,4
3,TENACIA,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462,ITALIA,199,6
4,HEDY LAMARR,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422,CHIPRE,186,7


In [92]:
df_transito.drop_duplicates()

,barco,origen,destino,llegada,salida,alineacion,consignatorio,gt,escala,bandera,eslora,calado
0,,,,,,,,,,,,
1,VALIANT LADY,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192,BAHAMAS,277,8
2,ELEANOR ROOSEVELT,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066,CHIPRE,123,4
3,TENACIA,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462,ITALIA,199,6
4,HEDY LAMARR,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422,CHIPRE,186,7
...,...,...,...,...,...,...,...,...,...,...,...,...
1045,CIUDAD DE ALCUDIA,Valencia,Valencia,31/08/2023 06:57,31/08/2023 11:25,1ER. TRAMO EXT MLLES CIALES. (del 12 al 2),"GRIMALDI LOGISTICA ESPAÑA, SL",33.588,3398,ESPAÑA,183,7
1046,YACHT SERVANT,GENOA,PORT EVERGLADES,31/08/2023 08:57,02/09/2023 16:05,M. 2ª ALIN. PONIENTE SUR (del 18 al 2),MIGUEL PUIGSERVER S A,44.522,3408,PAÍSES BAJOS,214,5
1047,ABSA UNO,SAN PEDRO DEL PINATAR,ALMERIA,31/08/2023 10:30,31/08/2023 18:50,MUELLE ADOSADO (del 16 al 12),ESTELA SHIPPING PALMA SA,317,3353,ESPAÑA,37,5
1048,ECO ADRIATICA,CAGLIARI,SAGUNTO,31/08/2023 13:30,31/08/2023 15:28,1ER. TRAMO EXT MLLES CIALES. (del 15 al 2),"GRIMALDI LOGISTICA ESPAÑA, SL",67.311,3338,ITALIA,238,7


In [93]:
df_clean = df_transito.copy()
df_clean.head()

,barco,origen,destino,llegada,salida,alineacion,consignatorio,gt,escala,bandera,eslora,calado
0,,,,,,,,,,,,
1,VALIANT LADY,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192,BAHAMAS,277,8
2,ELEANOR ROOSEVELT,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066,CHIPRE,123,4
3,TENACIA,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462,ITALIA,199,6
4,HEDY LAMARR,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422,CHIPRE,186,7


Perfecto. Tenemos un dataframe limpio con todas las incidencias de buques que han llegado y salido de puertos en el Mediterráneo entre los meses de junio y agosto junto con su información.
* Procedemos a pasar el dataframe a un csv para almacenarlo.

In [94]:
df_clean.to_csv('../data/transit_clean.csv', index=False)

In [5]:
# Voy a bajarme el csv de nuevo y pasarlo con indices de cara a que en SQL cada tabla tenga su indice y así poder usarlos de PK's.
df_index = pd.read_csv('../data/transit_clean.csv')
df_index.head()

,barco,origen,destino,llegada,salida,alineacion,consignatorio,gt,escala,bandera,eslora,calado
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VALIANT LADY,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192.0,BAHAMAS,277.0,8.0
2,ELEANOR ROOSEVELT,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066.0,CHIPRE,123.0,4.0
3,TENACIA,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462.0,ITALIA,199.0,6.0
4,HEDY LAMARR,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422.0,CHIPRE,186.0,7.0


In [6]:
df_index.to_csv('../data/transit_clean.csv', index=True)

In [10]:
driver.quit()